## Loading required libraries

In [ ]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

## Creating a Bucket

In [5]:
bucket_name = 'first1project'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [6]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':                
        s3.create_bucket(Bucket = bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [7]:
#set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://first1project/xgboost-as-a-built-in-algo/output


In [8]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')  # Taking data from the webpage
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [10]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac =1 , random_state = 179), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)  # Rows and Columns in Test and Train dataset

(28831, 61) (12357, 61)


In [17]:
sagemaker = boto3.client('sagemaker')
dir(sagemaker)

['_PY_TO_OP_NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_client_config',
 '_convert_to_request_dict',
 '_emit_api_params',
 '_endpoint',
 '_exceptions',
 '_exceptions_factory',
 '_get_waiter_config',
 '_load_exceptions',
 '_loader',
 '_make_api_call',
 '_make_request',
 '_register_handlers',
 '_request_signer',
 '_response_parser',
 '_serializer',
 '_service_model',
 'add_association',
 'add_tags',
 'associate_trial_component',
 'can_paginate',
 'create_action',
 'create_algorithm',
 'create_app',
 'create_app_image_config',
 'create_artifact',
 'create_auto_ml_job',
 'create_code_repository',
 'create_compilation_job',
 'create_c

In [24]:
#saving train and test into buckets

import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis = 1)], axis =1).to_csv('train.csv', index = False, header = False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
S3_train = sagemaker.TrainingInput(s3_data ='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [25]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis = 1)], axis =1).to_csv('test.csv', index = False, header = False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
S3_test = sagemaker.TrainingInput(s3_data ='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

# Building Xgboost model

In [26]:
container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version = '1.0-1') # AWS Sagemaker in-built Xgboost model

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [38]:
hyperparameters = {                          # Defined the hyperparameters which best suits the model. Excecuted Hyperparameter tuning  
                                             # on a local machine several times and the best output is considered
    "max_depth" : "5",
    "eta" : "0.2",
    "gamma" : "4",
    "min_child_weight" : "6",
    "subsample" : "0.7",
    "objective" : "binary:logistic",
    "num_round" : 50
}

In [39]:
estimator = sagemaker.estimator.Estimator(image_uri = container, 
                                         hyperparameters = hyperparameters,
                                         role = sagemaker.get_execution_role(),
                                         instance_count = 1,
                                         instance_type = 'ml.m5.2xlarge',
                                         volume_size = 5,
                                         output_path = output_path,
                                         use_spot_instances = True,
                                         max_run = 300,
                                         max_wait = 600)

In [40]:
estimator.fit({'train': S3_train, 'validation': S3_test})

2021-01-30 03:00:04 Starting - Starting the training job...
2021-01-30 03:00:28 Starting - Launching requested ML instancesProfilerReport-1611975604: InProgress
......
2021-01-30 03:01:29 Starting - Preparing the instances for training......
2021-01-30 03:02:31 Downloading - Downloading input data
2021-01-30 03:02:31 Training - Downloading the training image...
2021-01-30 03:03:00 Uploading - Uploading generated training model
2021-01-30 03:03:00 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is

## Deploying the Model

In [46]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type = "ml.m4.xlarge")

-----------------!

In [49]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_types = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [50]:
predictions_array

array([0.02503077, 0.04527099, 0.15711819, ..., 0.07941107, 0.02970827,
       0.03030096])

## Confusion Matrix

In [51]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.0%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10818)    38% (180)
Purchase        9% (1061)     62% (298) 



## Deleting the Endpoints

In [80]:
sagemaker.predictor.Predictor("sagemaker-xgboost-2021-01-30-03-12-10-800")
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[]